In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime as dt
import sys
sys.path.append('/Users/dnowacki/Documents/Grand Bay/py')
sys.path.append('/Users/dnowacki/Documents/python')
sys.path.append('/Users/dnowacki/Documents/stglib')
import stglib
import gbts
import shutil
import plotly
import plotly.graph_objs as go
import pandas as pd
import xarray as xr
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
gndcrmet = gbts.read_nerrs('/Volumes/Backstaff/field/gb/gndnerr/gndcrmet/GNDCRMET.csv')
basedir = '/Volumes/Backstaff/field/gb_proc/'

In [ ]:
def load_raw(site, dep):
    if site == '1080':
        height = '1'
    else:
        height = '3'
    fildir = basedir + site + dep + '/' + site + height + dep.upper() + 'dw/'

    RAW = {}
    print(fildir + site + height + dep.upper() + 'dw-raw.cdf')
    RAW = xr.open_dataset(fildir + site + height + dep.upper() + 'dw-raw.cdf')
    
    return RAW, fildir

def load_clean(site, dep, basedir):
    if site == '1080':
        height = '1'
    else:
        height = '3'
        
    fildir = basedir + site + dep + '/' + site + height +  dep.upper() + 'dw/'

    RAW = {}
    print(fildir + site + '1' + dep.upper() + 'dws-a.nc')
    RAW = xr.open_dataset(fildir + site + height + dep.upper() + 'dws-a.nc', decode_times=False)
    RAW['time'] = RAW['time_cf']
    RAW = RAW.drop('time2')
    RAW = xr.decode_cf(RAW)
    
    return RAW

In [ ]:
for site, dep, offset in zip(['1076', '1077', '1078', '1078', '1079', '1080'], 
                             ['b',    'b',    'a',    'b',    'a',    'b'], 
                             [0.25,   0.20,   0.52,   0.56,   0.20,   0.22]):
    RAW, fildir = load_raw(site, dep)
    
    met = gndcrmet['BP']/100
    met = met.rename('atmpres')
    met = met.reindex(time=RAW['time'], copy=True, method='nearest')
    met.attrs.update(offset=offset)
    met.to_netcdf(fildir + 'atmpres.cdf')

In [ ]:
# this cell is where we set the offset!

trace = go.Scatter(
    x = RAW['time'].values,
    y = RAW['P_1'][:,0].values - met.values - 0.22, 
    name = 'P_1 - met'
)
data = [trace]
fig = go.Figure(data=data)
plotly.offline.iplot(fig)

In [ ]:
site = '1078'
dep = 'a'

ds = load_clean(site, dep, basedir)
fig = plotly.tools.make_subplots(rows=2, cols=1, shared_xaxes=True, print_grid=False)
trace = go.Scatter(
    x = ds['time'].to_series(),
    y = ds['wh_4061'].to_series(), 
    name = ds['wh_4061'].attrs['long_name']
)

trace2 = go.Scatter(
    x = ds['time'].to_series(),
    y = ds['wp_peak'].to_series(),
    name = ds['wp_peak'].attrs['long_name']
)
fig.append_trace(trace, 1, 1)
fig.append_trace(trace2, 2, 1)
trace2 = go.Scatter(
    x = ds['time'].to_series(),
    y = ds['wp_4060'].to_series(),
    name = ds['wp_4060'].attrs['long_name']
)
fig.append_trace(trace2, 2, 1)
# layout = go.Layout(title=site + dep + ' offset: ' + str(offset))
# data = [trace, trace2]
# fig = go.Figure(data=data)
plotly.offline.iplot(fig)

In [ ]:
fig = plotly.tools.make_subplots(rows=1, cols=1, shared_xaxes=True, print_grid=False)
trace = go.Scatter(x = ds['time'].to_series(),
    y = ds['wp_peak'].to_series() / ds['wp_4060'].to_series())

fig.append_trace(trace, 1, 1)

trace = go.Scatter(x = ds['time'].to_series(),
    y = ds['wp_peak'].to_series())

fig.append_trace(trace, 1, 1)
plotly.offline.iplot(fig)